# Desafio 2
# =========
* Request to Meli API

In [5]:
import os
import requests
import json

In [80]:
ACCESS_TOKEN = os.getenv('ACCESS_TOKEN')

headers = {
    'Authorization': f"Bearer {ACCESS_TOKEN}",
}
PARAMS = {'category':'MLA1000'}

response = requests.get('https://api.mercadolibre.com/sites/MLA/search?', headers=headers, params = PARAMS)
response

<Response [200]>

In [82]:
#print(response.json())

In [86]:
os.mkdir('get_MLA1000_json202201')
folder = './get_MLA1000_json202201/'

data = response.json()
with open(folder + 'data.json', 'w') as f:
    json.dump(data, f)

### From Pyspark

In [87]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

import requests
import json

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [89]:
def executeRestApi(verb, url, headers):
  #
  headers = {
      'content-type': "application/json",
      'Authorization': f"Bearer {ACCESS_TOKEN}"
  }
  PARAMS = {'category':'MLA1000'}
  res = None
  # Make API request, get response object back, create dataframe from above schema.
  try:
    if verb == "get":
      res = requests.get('https://api.mercadolibre.com/sites/MLA/search?', headers=headers, params = PARAMS)
      print(res)
    else:
      res = requests.get('https://api.mercadolibre.com/sites/MLA/search?', headers=headers, params = PARAMS)
  except Exception as e:
    return e
  if res != None and res.status_code == 200:
    return json.loads(res.text)
  return None

In [92]:
respuesta = executeRestApi('get', 'https://api.mercadolibre.com/sites', headers)

<Response [200]>


In [109]:
respuesta['results']

[{'id': 'MLA877456895',
  'site_id': 'MLA',
  'title': '4 Pilas Aaa Recargables Con Cargador Usb Para Pilas Aa Y Aaa',
  'seller': {'id': 417972986,
   'permalink': 'http://perfil.mercadolibre.com.ar/PL+MLA',
   'registration_date': '2019-03-19T08:41:57.000-04:00',
   'car_dealer': False,
   'real_estate_agency': False,
   'tags': ['brand', 'large_seller', 'mshops', 'messages_as_seller'],
   'seller_reputation': {'power_seller_status': 'platinum',
    'level_id': '5_green',
    'metrics': {'cancellations': {'period': '60 days', 'rate': 0, 'value': 0},
     'claims': {'period': '60 days', 'rate': 0.0044, 'value': 305},
     'delayed_handling_time': {'period': '60 days', 'rate': 0, 'value': 0},
     'sales': {'period': '60 days', 'completed': 64753}},
    'transactions': {'canceled': 25559,
     'period': 'historic',
     'total': 420401,
     'ratings': {'negative': 0.03, 'neutral': 0.01, 'positive': 0.96},
     'completed': 394842}}},
  'price': 1669,
  'prices': {'id': 'MLA877456895',

In [117]:
data = []
for item in respuesta['results']:
    lista = list()
    lista.append(item['id'])
    lista.append(item['site_id'])
    lista.append(item['seller']['id'])
    lista.append(int(item['price'])) #Resuelvo el error: field price: Can not merge type <class 'pyspark.sql.types.LongType'> and <class 'pyspark.sql.types.DoubleType'>
    data.append(lista)

In [119]:
mlColumns = ["id","site_id", "seller_id", "price"]
ml_df = spark.createDataFrame(data=data, schema = mlColumns)
ml_df.printSchema()
ml_df.show(truncate=False)

root
 |-- id: string (nullable = true)
 |-- site_id: string (nullable = true)
 |-- seller_id: long (nullable = true)
 |-- price: long (nullable = true)

+-------------+-------+---------+-----+
|id           |site_id|seller_id|price|
+-------------+-------+---------+-----+
|MLA877456895 |MLA    |417972986|1669 |
|MLA877436060 |MLA    |417972986|26999|
|MLA904369062 |MLA    |268317904|2719 |
|MLA903838918 |MLA    |671347056|1839 |
|MLA903840517 |MLA    |671347056|5819 |
|MLA1103327815|MLA    |85719635 |3499 |
|MLA903321020 |MLA    |666749750|5199 |
|MLA918462579 |MLA    |149939279|4455 |
|MLA727049560 |MLA    |296728610|641  |
|MLA905508180 |MLA    |185702920|32559|
|MLA1116961065|MLA    |164352247|8999 |
|MLA832856590 |MLA    |52130335 |10889|
|MLA1111761158|MLA    |10119080 |2549 |
|MLA929706071 |MLA    |256189964|11399|
|MLA927538053 |MLA    |269998830|3999 |
|MLA924363879 |MLA    |296728610|5604 |
|MLA1106913910|MLA    |372556275|6190 |
|MLA1118718859|MLA    |256324891|30499|
|MLA876